# Vertex Tabular Binary Classification with .CustomJob() / Container

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/jchavezar/vertex-ai-samples/blob/main/vertex-custom-ml/sklearn/container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
</table>

**Use case: predict if a customer will buy on return visit.**

The ecommerce dataset has different training features:
- latest_ecommerce_progress
- bounces
- time_on_site
- pageviews
- source
- medium
- channel_grouping
- device_category
- country

The label: will_buy_on_return_visit

Data is imbalanced

## Colab Only

*Uncomment and execute if colab*

In [ ]:
#! pip3 install --upgrade google-cloud-aiplatform -q google-cloud-bigquery db-dtypes

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

## Import Libraries

In [3]:
from google.cloud import aiplatform as aip

## Variables

In [4]:
IMAGE_TRAIN_URI = "gcr.io/jchavezar-demo/sklearn-train"
IMAGE_PREDICTION_URI = "gcr.io/jchavezar-demo/ecommerce:fast-onnx"
MODEL_URI = "gs://vtx-models/ecommerce/sklearn"

## Initialize Vertex Project

*Remember to change project and staging_bucket*

In [6]:
aip.init(project='jchavezar-demo', staging_bucket='gs://vtx-staging')


## Train your Model using Vertex (Containers)

In [9]:
worker_pool_specs=[
    {
        "machine_spec": {
            "machine_type": "n1-standard-4"
        },
        "replica_count" : 1,
        "container_spec": {
            "image_uri": IMAGE_TRAIN_URI
        }
    }
]

my_job = aip.CustomJob(
    display_name = "sklearn-customjob-train",
    worker_pool_specs = worker_pool_specs,
    base_output_dir = MODEL_URI,
)

my_job.run()

/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating CustomJob
CustomJob created. Resource name: projects/569083142710/locations/us-central1/customJobs/746408553757016064
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/569083142710/locations/us-central1/customJobs/746408553757016064')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/746408553757016064?project=569083142710
CustomJob projects/569083142710/locations/us-central1/customJobs/746408553757016064 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/746408553757016064 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/746408553757016064 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/746408553757016064 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/74640855375701

## Upload Model to Model Registry

In [10]:
model = aip.Model.upload(
    display_name='sklearn-ecommerce-1',
    artifact_uri=MODEL_URI,
    serving_container_image_uri=IMAGE_PREDICTION_URI,
    serving_container_predict_route='/predict',
    serving_container_health_route='/health'
)

/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating Model
Create Model backing LRO: projects/569083142710/locations/us-central1/models/6683396822599204864/operations/6213083816194473984


/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/sockcop/.local/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accoun

Model created. Resource name: projects/569083142710/locations/us-central1/models/6683396822599204864@1
To use this Model in another session:
model = aiplatform.Model('projects/569083142710/locations/us-central1/models/6683396822599204864@1')


## Deploy Model for Online Prediction (Vertex Endpoints)

In [11]:
endpoint = model.deploy(
    deployed_model_display_name='sklearn-ecommerce',
    machine_type='n1-standard-2',
    min_replica_count=1,
    max_replica_count=1
)

Creating Endpoint
Create Endpoint backing LRO: projects/569083142710/locations/us-central1/endpoints/1510944480843268096/operations/1903138972800909312
Endpoint created. Resource name: projects/569083142710/locations/us-central1/endpoints/1510944480843268096
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/569083142710/locations/us-central1/endpoints/1510944480843268096')
Deploying model to Endpoint : projects/569083142710/locations/us-central1/endpoints/1510944480843268096
Deploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/1510944480843268096/operations/8345538239754403840


FailedPrecondition: 400 Model server terminated: model server container terminated: exit_code: 	 1
reason: "Error"
started_at {
  seconds: 1677298024
}
finished_at {
  seconds: 1677298026
}
. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=569083142710&resource=aiplatform.googleapis.com%252FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%221510944480843268096%22%0Aresource.labels.location%3D%22us-central1%22.

## Delete and Clean Components

In [ ]:
endpoint.undeploy()
model.delete